In [1]:
import snap 
import csv
import pandas as pd
import random
import networkx as nx
from node2vec import Node2Vec
#from deepwalk import DeepWalk
#from graphembedding.sdne import SDNE
#from gem.embedding.hope import HOPE

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
from GraphEmbedding.ge import LINE 
from joblib import parallel_config
from sklearn.model_selection import ParameterGrid

from scipy.sparse import coo_matrix


%run Fairness.ipynb
%run Embeddings_Classification.ipynb


https://snap.stanford.edu/data/twitch_gamers.html

read pathas as Graph and Dataframe

In [2]:
path_edges = 'Datasets/Twitch/large_twitch_edges.csv'
path_target = 'Datasets/Twitch/large_twitch_features.csv'

In [3]:
df_graph = pd.read_csv(path_edges)
G = nx.from_pandas_edgelist(df_graph, 'numeric_id_1', 'numeric_id_2')

In [4]:
print(G.number_of_nodes())
print(G.number_of_edges())

168114
6797557


add nodes to a list

In [5]:
list_nodes = []
for nodes in G.nodes():
    list_nodes.append(nodes)
random.seed(42)

# Perform random sampling to get 1000 random nodes (because dataset to big)
sampled_nodes = random.sample(list_nodes, 1000)
print(len(sampled_nodes))


1000


In [6]:
df = pd.read_csv(path_target)
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [7]:
df.head()

,views,mature,life_time,created_at,updated_at,numeric_id,dead_account,language,affiliate
0,7879,1,969,2016-02-16,2018-10-12,0,0,EN,1
1,500,0,2699,2011-05-19,2018-10-08,1,0,EN,0
2,382502,1,3149,2010-02-27,2018-10-12,2,0,EN,1
3,386,0,1344,2015-01-26,2018-10-01,3,0,EN,0
4,2486,0,1784,2013-11-22,2018-10-11,4,0,EN,0


categorize the attribute values in form of integers

In [8]:
df['created_at']  = pd.to_datetime(df['created_at'])
df['created_at']= df['created_at'].dt.year
df['updated_at'] = pd.to_datetime(df['updated_at'])
df['updated_at'] = df['updated_at'].dt.year
df['language'] = pd.factorize(df['language'])[0]


In [9]:
df.head()

,views,mature,life_time,created_at,updated_at,numeric_id,dead_account,language,affiliate
0,7879,1,969,2016,2018,0,0,0,1
1,500,0,2699,2011,2018,1,0,0,0
2,382502,1,3149,2010,2018,2,0,0,1
3,386,0,1344,2015,2018,3,0,0,0
4,2486,0,1784,2013,2018,4,0,0,0


In [10]:
print(sampled_nodes[0])

24573


Create network based on the sampled nodes

In [11]:
G_nx = nx.Graph()
for node in sampled_nodes:
        G_nx.add_node(node)
print(G_nx.number_of_nodes())

1000


Add edges to the new network G_nx

In [12]:
for node in G_nx.nodes():
    node_x_neighbors = set() 
    for neighbor in G.neighbors(node):
        node_x_neighbors.add(neighbor)
    for nd in node_x_neighbors:
        if node != nd and G_nx.has_node(nd):
                G_nx.add_edge(node, nd)


In [13]:
print(G_nx.number_of_edges())

287


In [14]:
print(nx.is_connected(G_nx))

False


Find the largest connected components and create a subgraph based on that

In [15]:
largest_cc = max(nx.connected_components(G_nx), key=len)
G_new = G_nx.subgraph(largest_cc)

In [16]:
print(nx.is_connected(G_new))

True


In [17]:
print(G_new.number_of_nodes())
print(G_new.number_of_edges())

191
215


In [18]:
# Resetting the index of the nodes from 0 to numberr of ndes
new_indices = range(0, G_new.number_of_nodes())
mapping = {old_index: new_index for new_index, old_index in zip(new_indices, G_new.nodes())}
G_new = nx.relabel_nodes(G_new, mapping)

In [19]:
print(G_new.number_of_nodes())
print(G_new.number_of_edges())

191
215


In [20]:
list_nodes = [nodes for nodes in G_new.nodes()]
print(len(list_nodes))

191


In [21]:
df_new= df.loc[list_nodes]
# df_views = []
# df_mature = []
# df_created_at = []
# df_updatet_at = []
# df_dead_account = []
# df_language = []


df_target = df_new['affiliate']
df_views = df_new['views']
df_mature = df_new['mature']
df_created_at = df_new['created_at']
# Replace dates with numeric values (e.g., year, month, day)
df_updatet_at = df_new['updated_at']
df_dead_account = df_new['dead_account']
df_language = df_new['language']

In [22]:
print(df_language)

0      0
1      0
2      0
3      0
4      0
      ..
186    0
187    1
188    1
189    0
190    9
Name: language, Length: 191, dtype: int64


Add attributes to the new network

In [23]:
'add synthetic gender '
list_gender = {i: i % 2 for i in range(G_new.number_of_nodes())}
print(len(list_gender))
nx.set_node_attributes(G_new, list_gender, 'gender')
nx.set_node_attributes(G_new, df_views, 'views')
nx.set_node_attributes(G_new, df_mature, 'mature')
nx.set_node_attributes(G_new, df_created_at, 'created_at')
nx.set_node_attributes(G_new, df_updatet_at, 'updated_at')
nx.set_node_attributes(G_new, df_dead_account, 'dead_account')
nx.set_node_attributes(G_new, df_language, 'language')
print(G_new.nodes[list_nodes[0]])

191
{'gender': 0, 'views': 7879, 'mature': 1, 'created_at': 2016, 'updated_at': 2018, 'dead_account': 0, 'language': 0}


In [24]:
for i in G_new.nodes:
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


In [25]:
import pandas as pd
from scipy.sparse import coo_matrix

# Create the COO sparse matrix
df_features = df.apply(pd.to_numeric, errors='coerce')
row_indices, col_indices = np.where(~np.isnan(df_new.values))
print(row_indices)
print(col_indices)

coo_matrix = coo_matrix((df_new.values.ravel(), (row_indices, col_indices)), shape=(df_new.shape[0], df_new.shape[1]))

# Print the COO sparse matrix
print(coo_matrix)

[  0   0   0 ... 190 190 190]
[0 1 2 ... 6 7 8]
  (0, 0)	7879
  (0, 1)	1
  (0, 2)	969
  (0, 3)	2016
  (0, 4)	2018
  (0, 5)	0
  (0, 6)	0
  (0, 7)	0
  (0, 8)	1
  (1, 0)	500
  (1, 1)	0
  (1, 2)	2699
  (1, 3)	2011
  (1, 4)	2018
  (1, 5)	1
  (1, 6)	0
  (1, 7)	0
  (1, 8)	0
  (2, 0)	382502
  (2, 1)	1
  (2, 2)	3149
  (2, 3)	2010
  (2, 4)	2018
  (2, 5)	2
  (2, 6)	0
  :	:
  (188, 2)	721
  (188, 3)	2016
  (188, 4)	2018
  (188, 5)	188
  (188, 6)	0
  (188, 7)	1
  (188, 8)	0
  (189, 0)	6525
  (189, 1)	0
  (189, 2)	1647
  (189, 3)	2014
  (189, 4)	2018
  (189, 5)	189
  (189, 6)	0
  (189, 7)	0
  (189, 8)	0
  (190, 0)	2850
  (190, 1)	0
  (190, 2)	632
  (190, 3)	2017
  (190, 4)	2018
  (190, 5)	190
  (190, 6)	0
  (190, 7)	9
  (190, 8)	0


In [26]:
# Resetting the index of the nodes from 0 to numberr of ndes
new_indices = range(0, G_new.number_of_nodes())
mapping = {old_index: new_index for new_index, old_index in zip(new_indices, G_new.nodes())}
nx_Graph = nx.relabel_nodes(G_new, mapping)

In [28]:
# Parameter
parameter_grid = {
    'dimensions': [16, 32],
    'walk_length': [20, 30],
    'walk_number': [25, 75],
    'workers': [2, 4]
}

#apply SINE embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_w,best_fpr_m, dataframe = grid_search_SINE(nx_Graph, coo_matrix, parameter_grid,  list(df_target), list(list_gender.values()))

done create embeding vectors
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0]
predic
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
[[16  4]
 [11  1]]
[[13  3]
 [ 9  1]]
done iteration
[[16  4]
 [11  1]]
[[13  3]
 [ 9  1]]
PPV_f and PPV_m is equal: False
[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0]
predic
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
[[16  4]
 [11  1]]
[[13  3]
 [ 9  1]]
[[16  4]
 [11  1]]


done create embeding vectors
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0]
predic
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
[[16  4]
 [11  1]]
[[13  3]
 [ 9  1]]
done iteration
[[16  4]
 [11  1]]
[[13  3]
 [ 9  1]]
PPV_f and PPV_m is equal: False
[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0]
predic
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
[[16  4]
 [11  1]]
[[13  3]
 [ 9  1]]
[[16  4]
 [11  1]]


In [29]:
print(best_params, best_fpr_w,best_fpr_m, dataframe)

{'dimensions': 16, 'walk_length': 20, 'walk_number': 25, 'workers': 4} 0.1 0.125                              Female      Male
Group Fairness             0.428571  0.571429
Predictive Parity          0.333333  0.500000
False Positive Error Rate  0.100000  0.125000
Equalized Opportunity      0.916667  0.800000
Equalized Odds PPV         0.083333  0.200000
Equalized Odds FPR         0.100000  0.125000


In [30]:
#parameter
parameter_grid = {
    'dimensions': [16, 32],
    'walk_length': [20, 30],
    'walk_number': [25, 75],
    'workers': [2, 4]
}

#apply AE embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_w,best_fpr_m, dataframe = grid_search_AE(G_new, coo_matrix, parameter_grid,  list(df_target), list(list_gender.values()))

done create embeding vectors
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0]
predic
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
[[15  5]
 [10  2]]
[[14  2]
 [ 9  1]]
done iteration
[[15  5]
 [10  2]]
[[14  2]
 [ 9  1]]
PPV_f and PPV_m is equal: False
[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0]
predic
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
[[15  5]
 [10  2]]
[[14  2]
 [ 9  1]]
[[15  5]
 [10  2]]


done create embeding vectors
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0]
predic
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
[[16  4]
 [11  1]]
[[14  2]
 [ 9  1]]
done iteration
[[16  4]
 [11  1]]
[[14  2]
 [ 9  1]]
PPV_f and PPV_m is equal: False
[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0]
predic
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
[[16  4]
 [11  1]]
[[14  2]
 [ 9  1]]
[[16  4]
 [11  1]]


In [32]:
print(best_params, best_fpr_w,best_fpr_m, dataframe)

{'dimensions': 16, 'walk_length': 20, 'walk_number': 75, 'workers': 2} 0.1 0.0                              Female  Male
Group Fairness             0.800000   0.2
Predictive Parity          0.500000   1.0
False Positive Error Rate  0.100000   0.0
Equalized Opportunity      0.833333   0.9
Equalized Odds PPV         0.166667   0.1
Equalized Odds FPR         0.100000   0.0


In [33]:
#parameter
parameter_grid = {
    'dimensions': [16, 32],
    'walk_length': [20, 30],
    'num_walks': [25, 75],
    'workers': [2, 4]
}

#apply Node2Vec embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_w, best_fpr_m,  dataframe = perform_grid_search(G_new, parameter_grid, list(df_target), list(list_gender.values()))


Computing transition probabilities:   0%|          | 0/191 [00:00<?, ?it/s]

done create embeding vectors
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0]
predic
[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
[[14  6]
 [ 8  3]]
[[14  7]
 [ 4  2]]
done iteration
[[14  6]
 [ 8  3]]
[[14  7]
 [ 4  2]]
PPV_f and PPV_m is equal: False
[0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0]
predic
[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
[[14  6]
 [ 8  3]]
[[14  7]
 [ 4  2]]
[[14  6]
 [ 8  3]]
[[14  7]
 [ 

Computing transition probabilities:   0%|          | 0/191 [00:00<?, ?it/s]

done create embeding vectors
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0]
predic
[0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[[15  5]
 [ 9  1]]
[[16  6]
 [ 5  1]]
done iteration
[[15  5]
 [ 9  1]]
[[16  6]
 [ 5  1]]
PPV_f and PPV_m is equal: False
[1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0]
predic
[0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[[15  5]
 [ 9  1]]
[[16  6]
 [ 5  1]]
[[15  5]
 [ 9  1]]
[[16  6]
 [ 5  1]]
[[15 

Computing transition probabilities:   0%|          | 0/191 [00:00<?, ?it/s]

...

done create embeding vectors
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]
predic
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[[17  2]
 [10  1]]
[[16  2]
 [ 9  1]]
done iteration
[[17  2]
 [10  1]]
[[16  2]
 [ 9  1]]
PPV_f and PPV_m is equal: True
[0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]
predic
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[[17  2]
 [10  1]]
[[16  2]
 [ 9  1]]
[[17  2]
 [10  1]]
[[16  2]
 [ 9  1]]
[[17  

Computing transition probabilities:   0%|          | 0/191 [00:00<?, ?it/s]

done create embeding vectors
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
predic
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
[[15  2]
 [12  2]]
[[13  3]
 [11  0]]
done iteration
[[15  2]
 [12  2]]
[[13  3]
 [11  0]]
PPV_f and PPV_m is equal: False
[0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
predic
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
[[15  2]
 [12  2]]
[[13  3]
 [11  0]]
[[15  2]
 [12  2]]
[[13  3]
 [1

Computing transition probabilities:   0%|          | 0/191 [00:00<?, ?it/s]

done create embeding vectors
[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0]
predic
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[[14  3]
 [ 9  1]]
[[19  1]
 [10  1]]
done iteration
[[14  3]
 [ 9  1]]
[[19  1]
 [10  1]]
PPV_f and PPV_m is equal: False
[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0]
predic
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[[14  3]
 [ 9  1]]
[[19  1]
 [10  1]]
[[14  3]
 [ 9  1]]
[[19  1]
 [10  1]]
[[14  3]
 [ 9  1]]
[[19  1]
 [10  1]]
fpF

In [34]:
print(f"Best Parameters: {best_params}")
print(f"Best Fpr: {best_fpr_w}")
print(f"Best Fpr: {best_fpr_m}")
print(dataframe)

Best Parameters: {'dimensions': 16, 'num_walks': 25, 'walk_length': 30, 'workers': 4}
Best Fpr: 0.1111111111111111
Best Fpr: 0.0
                             Female      Male
Group Fairness             0.666667  0.333333
Predictive Parity          0.500000  1.000000
False Positive Error Rate  0.111111  0.000000
Equalized Opportunity      0.818182  0.857143
Equalized Odds PPV         0.181818  0.142857
Equalized Odds FPR         0.111111  0.000000


In [38]:
# Parameter
parameter_grid = {
    'window': [16, 32],
    'walk_length': [20, 30],
    'number_of_random_walks': [25, 75],
}

#apply DeepWalk embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_f,best_fpr_m, dataframe =  perform_grid_search_DeepWalk(G_new, parameter_grid, df_target, list(list_gender.values()))

25
20
16


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 191/191 [00:00<00:00, 693.70it/s]


[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0]
predic
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[13  0]
 [11  2]]
[[16  1]
 [14  1]]
fpF: 0.0
fpM: 0.058823529411764705
[[13  0]
 [11  2]]
[[16  1]
 [14  1]]
PPV_f and PPV_m is equal: False
[0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0]
predic
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[13  0]
 [11  2]]
[[16  1]
 [14  1]]
[[13  0]
 [11  2]]
[[16  1]
 [14  1]]
[[13  0]
 [11  2]]
[[16  1]
 [14  1]]
25
20
32


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 191/191 [00:00<00:00, 660.85it/s]


[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[[15  1]
 [11  1]]
[[17  3]
 [ 8  2]]
fpF: 0.0625
fpM: 0.15
[[15  1]
 [11  1]]
[[17  3]
 [ 8  2]]
PPV_f and PPV_m is equal: False
[1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[[15  1]
 [11  1]]
[[17  3]
 [ 8  2]]
[[15  1]
 [11  1]]
[[17  3]
 [ 8  2]]
[[15  1]
 [11  1]]
[[17  3]
 [ 8  2]]
25
30
16


...

[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[13  0]
 [11  0]]
[[20  1]
 [13  0]]
fpF: 0.0
fpM: 0.047619047619047616
[[13  0]
 [11  0]]
[[20  1]
 [13  0]]
PPV_f and PPV_m is equal: True
[0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[13  0]
 [11  0]]
[[20  1]
 [13  0]]
[[13  0]
 [11  0]]
[[20  1]
 [13  0]]
[[13  0]
 [11  0]]
[[20  1]
 [13  0]]
75
30
16


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 191/191 [00:00<00:00, 285.26it/s]


[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[[19  1]
 [10  2]]
[[14  1]
 [10  1]]
fpF: 0.05
fpM: 0.06666666666666667
[[19  1]
 [10  2]]
[[14  1]
 [10  1]]
PPV_f and PPV_m is equal: False
[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
predic
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[[19  1]
 [10  2]]
[[14  1]
 [10  1]]
[[19  1]
 [10  2]]
[[14  1]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 191/191 [00:00<00:00, 241.60it/s]


[1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1
 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[14  1]
 [15  0]]
[[18  0]
 [10  0]]
fpF: 0.06666666666666667
fpM: 0.0
[[14  1]
 [15  0]]
[[18  0]
 [10  0]]
PPV_f and PPV_m is equal: True
[1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[14  1]
 [15  0]]
[[18  0]
 [10  0]]
[[14  1]
 [15  0]]
[[18  0]
 [10  0]]
[[14  1]
 [15  0

In [39]:
print(f"Best Parameters: {best_params}")
print(f"Best Fpr: {best_fpr_f}")
print(f"Best Fpr: {best_fpr_m}")
print(dataframe)

Best Parameters: {'number_of_random_walks': 25, 'walk_length': 20, 'window': 16}
Best Fpr: 0.0
Best Fpr: 0.058823529411764705
                             Female      Male
Group Fairness             0.500000  0.500000
Predictive Parity          1.000000  0.500000
False Positive Error Rate  0.000000  0.058824
Equalized Opportunity      0.846154  0.933333
Equalized Odds PPV         0.153846  0.066667
Equalized Odds FPR         0.000000  0.058824


In [40]:
dataframe.head()

,Female,Male
Group Fairness,0.500000,0.500000
Predictive Parity,1.000000,0.500000
False Positive Error Rate,0.000000,0.058824
Equalized Opportunity,0.846154,0.933333
Equalized Odds PPV,0.153846,0.066667
